In [1]:
import math

In [2]:
class N_gram:
	def __init__(self):
		pass

	def n_gram(self, text, n):
		self.text = text.split()
		self.n = n
		Ngram = [tuple(self.text[index : index + self.n]) for index in range(0, len(self.text) - self.n + 1)]
		return Ngram

In [3]:
class BLEU(N_gram):
	def __init__(self):
		self.max_n_gram = 4

	def get_score(self, refs, pred):
		pred_lenght = len(pred.split())
		refs_lenght = []
		refs_ngram = list()
		for ref in refs:
			refs_lenght.append(len(ref.split()))
			result = [self._counter_ngrams(self.n_gram(ref, n)) for n in range(1 , self.max_n_gram + 1)]
			refs_ngram.append(result)
		pred_ngram = [self._counter_ngrams(self.n_gram(pred, n)) for n in range(1 , self.max_n_gram + 1)]

		score = self._get_bleu(refs_ngram, pred_ngram, pred_lenght, refs_lenght)
		return score


	def _get_bleu(self, refs_ngram, pred_ngram, pred_lenght, refs_lenght):
		clipped_precisions = []
		for number_ngram in range(0, self.max_n_gram):
			pred_gram = pred_ngram[number_ngram]
			refs_gram = [refs_ngram[num_refs_sents][number_ngram] for num_refs_sents in range(0, len(refs_ngram))]
			clipped_count = self._clipped_count(refs_gram, pred_gram)
			clipped_precision = clipped_count / (pred_lenght - number_ngram)
			clipped_precisions.append(clipped_precision)

		pb = self._brevity_penalty(refs_lenght, pred_lenght)
		db = 0
		for element in clipped_precisions:
			if element == 0:
				db += 0
			else:
				db += (math.log(element))
		db = (db / 4)
		if db == 0:
			return 0
		else:
			return pb * math.exp(db)

	def _clipped_count(self, refs_grams, pred_gram):
		count = 0
		for p_gram in pred_gram:
			counter_pred = pred_gram[p_gram]
			max_r_gram = 0
			for ref_gram in refs_grams:
				if p_gram in ref_gram:
					max_r_gram = max(max_r_gram, ref_gram[p_gram])
			count += min(max_r_gram, counter_pred)
		return count


	def _brevity_penalty(self, refs_lenght, pred_lenght):
		closest_ref_lenght = self._get_closest_ref_lenght(refs_lenght, pred_lenght)
		if closest_ref_lenght < pred_lenght:
			return 1
		elif closest_ref_lenght >= pred_lenght:
			return math.exp(1 - closest_ref_lenght / pred_lenght)

	def _get_closest_ref_lenght(self, ref_lenght, pred_lenght):
		return min(ref_lenght , key=lambda temp : abs(temp - pred_lenght))

	def _counter_ngrams(self, ngram):
		result = {}
		for element in ngram:
			if element in result:
				result[element] += 1
			elif element not in result:
				result[element] = 1
		return result


In [5]:
test = BLEU()
refs = [
    "The ball is  blue",
    "the color of ball is blue"
]
pred = "the ball is blue color"

print(f"the score of translate is {round(test.get_score(refs, pred) * 100, 2)}")

the score of translate is 63.89
